In [ ]:
!pip install evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00


In [ ]:
!pip install rouge


In [ ]:
import pandas as pd
import evaluate

In [ ]:
df = pd.read_excel('/content/code_t5_plus_eval_all_task_new_results.xlsx')


In [ ]:
df.columns

Index(['Query', 'Response', 'Generated_Response', 'Generation_Time (seconds)',
       'Task'],
      dtype='object')

In [ ]:
df = df[df['actual_event'].isna()==False]

In [ ]:
df

,Query,Response,Generated_Response,Generation_Time (seconds),Task
0,Create a relevance script that meets the provi...,"files whose (name of it ends with "".xlsx"") of ...","files whose (name of it ends with "".txt"") of f...",1.134329,relevance
1,Generate BigFix relevance code based on the gi...,"exists (elements of set of (""apple""; ""banana""...","exists (elements of set of (""apple""; ""banana""...",1.523320,relevance
2,Could you inspect the relevance code for error...,"pathname of regapp ""skype.exe""","pathname of regapp ""skype.exe""",0.737162,rectify
3,Formulate BigFix relevance logic for the given...,full name of user,full name of user,0.398506,relevance
4,Build a description for the given BigFix relev...,Verify if the operating system is not Windows ...,Check if the operating system name starts with...,10.904058,description
...,...,...,...,...,...
735,Construct a BigFix relevance statement that fi...,Win98|WinME|Win2000|WinXP|Win2003|WinXP-2003|W...,Win98|WinME|Win2000|WinXP|Win2003|WinXP-2003|W...,2.338367,relevance
736,Develop a description for the specified releva...,Check if the operating system is not a Windows...,Verify if the operating system starts with 'Wi...,7.796154,description
737,Formulate BigFix relevance logic for the given...,['(if( name of operating system starts with Wi...,['(if( name of operating system starts with Wi...,10.675630,relevance
738,Can you ensure that the relevance code is erro...,"exists event log ""DNS Server""","exists event log ""DNS Server""",0.560741,rectify


In [ ]:
import pandas as pd
from evaluate import load
import re



def calculate_bleu_scores(candidate, reference):
    # Load the BLEU evaluation metric
    bleu = load("bleu")

    # Remove leading/trailing whitespaces and extra spaces
    candidate = candidate.strip()
    reference = reference.strip()

    # Convert to lowercase (optional, depending on your preference)
    candidate = candidate.lower()
    reference = reference.lower()

    # Remove special characters using regular expressions
    candidate = re.sub(r'[^A-Za-z0-9 ]+', '', candidate)
    reference = re.sub(r'[^A-Za-z0-9 ]+', '', reference)

    print(f"Candidate: {candidate}")
    print(f"Reference: {reference}")


    # Compute the BLEU score for the given pair of candidate and reference
    result = bleu.compute(predictions=[candidate], references=[[reference]])

    print(f"BLEU Result: {result}")

    # Return the BLEU scores
    return result


In [ ]:
# Assuming df is your DataFrame
df['bleu_scores'] = df.apply(lambda row: calculate_bleu_scores(row['Generated_Response'], row['Response']), axis=1)

Candidate: files whose name of it ends with txt of folder aion
Reference: files whose name of it ends with xlsx of folder aion
BLEU Result: {'bleu': 0.7016879391277371, 'precisions': [0.9090909090909091, 0.8, 0.6666666666666666, 0.5], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 11, 'reference_length': 11}
Candidate: exists elements of set of apple banana orange whose length of it
Reference: exists elements of set of apple banana orange whose length of it
BLEU Result: {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 12, 'reference_length': 12}
Candidate: pathname of regapp skypeexe
Reference: pathname of regapp skypeexe
BLEU Result: {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 4, 'reference_length': 4}
Candidate: full name of user
Reference: full name of user
BLEU Result: {'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brev

In [ ]:
# Assuming df is your DataFrame with 'bleu_scores' column
df['bleu_score'] = df['bleu_scores'].apply(lambda x: x['bleu'])  # Assuming 'bleu' is the key for BLEU score

# Calculate the mean of BLEU scores
mean_bleu = df['bleu_score'].mean()

# Print the mean BLEU score
print("Mean BLEU Score:", mean_bleu)

Mean BLEU Score: 0.6883362976009405


In [ ]:
# Print individual BLEU scores for each r
df[['Generated_Response', 'Response', 'bleu_scores']]


,Generated_Response,Response,bleu_scores
0,"<Relevance>properties of type ""user""</Relevance>","<Relevance>properties of type ""user""</Relevance>","{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
1,<Relevance>'(if( name of operating system star...,<Relevance>'(if( name of operating system star...,"{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
2,<Relevance>(name of it = Win2012R2 OR name of ...,<Relevance>(name of it = Win2012R2 OR name of ...,"{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
3,"<Relevance>parent folder of file ""/VIT/passwd1...","<Relevance>parent folder of file ""/VIT/passwd1...","{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
4,"<Relevance>parent folder of file ""/btech/passw...","<Relevance>parent folder of file ""/btech/passw...","{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
...,...,...,...
603,comment of user,comment of user,"{'bleu': 0.0, 'precisions': [1.0, 1.0, 1.0, 0...."
604,(name of it = Win7 AND service pack major vers...,(name of it = Win7 AND service pack major vers...,"{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
605,"exists file ""passwd3"" of folder ""/btech""","exists file ""passwd3"" of folder ""/btech""","{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."
606,((name of it = Win7) AND service pack major ve...,((name of it = Win7) AND service pack major ve...,"{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1...."


In [ ]:
from rouge import Rouge

# Initialize Rouge
rouge = Rouge()

# Function to calculate ROUGE scores
def calculate_rouge_scores(row):
    scores = rouge.get_scores(row['Generated_Response'], row['Response'])
    return scores[0]  # Assuming you're interested in the first (and often most commonly used) set of ROUGE scores

# Apply the function to each row
df['rouge_scores'] = df.apply(calculate_rouge_scores, axis=1)




In [ ]:
# Assuming your 'rouge_scores' column contains dictionaries
df['rouge-1-f'] = df['rouge_scores'].apply(lambda x: x['rouge-1']['f'])
df['rouge-2-f'] = df['rouge_scores'].apply(lambda x: x['rouge-2']['f'])
df['rouge-l-f'] = df['rouge_scores'].apply(lambda x: x['rouge-l']['f'])

# Calculate the mean of 'f' scores
mean_rouge_1_f = df['rouge-1-f'].mean()
mean_rouge_2_f = df['rouge-2-f'].mean()
mean_rouge_l_f = df['rouge-l-f'].mean()

print("Mean Rouge-1 F-score:", mean_rouge_1_f)
print("Mean Rouge-2 F-score:", mean_rouge_2_f)
print("Mean Rouge-l F-score:", mean_rouge_l_f)



Mean Rouge-1 F-score: 0.8469314895969906
Mean Rouge-2 F-score: 0.7503888891703885
Mean Rouge-l F-score: 0.8386007469887313


In [ ]:
df = df.drop(columns = ['bleu_scores','rouge_scores'],axis=1)

In [ ]:
df

,Query,Response,Generated_Response,Generation_Time (seconds),Task,bleu_score,rouge-1-f,rouge-2-f,rouge-l-f
0,Create a relevance script that meets the provi...,"files whose (name of it ends with "".xlsx"") of ...","files whose (name of it ends with "".txt"") of f...",1.134329,relevance,0.701688,0.909091,0.818182,0.909091
1,Generate BigFix relevance code based on the gi...,"exists (elements of set of (""apple""; ""banana""...","exists (elements of set of (""apple""; ""banana""...",1.523320,relevance,1.000000,1.000000,1.000000,1.000000
2,Could you inspect the relevance code for error...,"pathname of regapp ""skype.exe""","pathname of regapp ""skype.exe""",0.737162,rectify,1.000000,1.000000,1.000000,1.000000
3,Formulate BigFix relevance logic for the given...,full name of user,full name of user,0.398506,relevance,1.000000,1.000000,1.000000,1.000000
4,Build a description for the given BigFix relev...,Verify if the operating system is not Windows ...,Check if the operating system name starts with...,10.904058,description,0.190904,0.480620,0.189349,0.434109
...,...,...,...,...,...,...,...,...,...
735,Construct a BigFix relevance statement that fi...,Win98|WinME|Win2000|WinXP|Win2003|WinXP-2003|W...,Win98|WinME|Win2000|WinXP|Win2003|WinXP-2003|W...,2.338367,relevance,1.000000,1.000000,1.000000,1.000000
736,Develop a description for the specified releva...,Check if the operating system is not a Windows...,Verify if the operating system starts with 'Wi...,7.796154,description,0.103251,0.484211,0.170543,0.442105
737,Formulate BigFix relevance logic for the given...,['(if( name of operating system starts with Wi...,['(if( name of operating system starts with Wi...,10.675630,relevance,1.000000,0.985075,0.977011,0.985075
738,Can you ensure that the relevance code is erro...,"exists event log ""DNS Server""","exists event log ""DNS Server""",0.560741,rectify,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.to_excel('code_t5_plus_all_task_new_eval_metric.xlsx',engine='openpyxl')

In [ ]:
df_1 = pd.read_excel('code_t5_plus_all_task_new_eval_metric.xlsx')

In [ ]:
df_des = df_1[df_1['Task'] == 'description']
df_rev = df_1[df_1['Task'] == 'relevance']
df_rec = df_1[df_1['Task'] == 'rectify']

In [ ]:
print("Mean Bleu F-score:",df_des['bleu_score'].mean())
print("Mean Rouge-1 F-score:", df_des['rouge-1-f'].mean())
print("Mean Rouge-2 F-score:", df_des['rouge-2-f'].mean())
print("Mean Rouge-l F-score:", df_des['rouge-l-f'].mean())

Mean Bleu F-score: 0.45182085330537336
Mean Rouge-1 F-score: 0.7060758219048037
Mean Rouge-2 F-score: 0.5400832148586205
Mean Rouge-l F-score: 0.6878416436745175


In [ ]:
print("Mean Bleu F-score:",df_rec['bleu_score'].mean())
print("Mean Rouge-1 F-score:", df_rec['rouge-1-f'].mean())
print("Mean Rouge-2 F-score:", df_rec['rouge-2-f'].mean())
print("Mean Rouge-l F-score:", df_rec['rouge-l-f'].mean())

Mean Bleu F-score: 0.970482794630981
Mean Rouge-1 F-score: 0.9914693305483463
Mean Rouge-2 F-score: 0.985013245033938
Mean Rouge-l F-score: 0.9914693305483463


In [ ]:
print("Mean Bleu F-score:",df_rev['bleu_score'].mean())
print("Mean Rouge-1 F-score:", df_rev['rouge-1-f'].mean())
print("Mean Rouge-2 F-score:", df_rev['rouge-2-f'].mean())
print("Mean Rouge-l F-score:", df_rev['rouge-l-f'].mean())

Mean Bleu F-score: 0.7591153380466936
Mean Rouge-1 F-score: 0.9028838101569124
Mean Rouge-2 F-score: 0.8228732635343373
Mean Rouge-l F-score: 0.8995629201000942
